# Analyze Product Sentiment

In [ ]:
! pip install turicreate
import turicreate

# Read product review data

In [ ]:
# Reading the data and creating an SFrame of the data
products = turicreate.SFrame.read_csv('../input/reviews-of-amazon-baby-products/amazon_baby.csv')
products

# Explore data

In [ ]:
products

In [ ]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

Now, I am creating a subset of words to create a classifier. Often, ML practitioners will throw out words they consider “unimportant” before training their model. This procedure can often be helpful in terms of accuracy. Here, we are going to throw out all words except for the very few above. Using so few words in our model will hurt our accuracy, but help us interpret what our classifier is doing. 

In [ ]:

selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']


# Building a sentiment classifier

## Build word count vectors

In [ ]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'] )

Our first goal is to create a column products[‘awesome’] where each row contains the number of times the word ‘awesome’ showed up in the review for the corresponding product, and 0 if the review didn’t show up. One way to do this is to look at the each row ‘word_count’ column and follow this logic: 

In [ ]:
# Loop through word counts to create a classifier for only a few words 
# Created an individual column for each item 
for word in selected_words:
    products[word] = products['word_count'].apply(lambda counts: counts.get(word, 0))

products

Using the .sum() method on each of the new columns you created, answer the following questions: Out of the selected_words, which one is most used in the dataset? Which one is least used?

In [ ]:
for word in selected_words:
    print("\nThe number of times {} appears: {}".format(word, products[word].sum()))

As we can see above, the highest count is of the word `great`
and the lowest count is of the word `wow`

## Train and Test Split

In [ ]:
train_data,test_data = products.random_split(.8, seed=0)

In [ ]:
# Features to be trained on 
features = selected_words

# Define what is positive and negative sentiment

In [ ]:
products['rating'].show()

In [ ]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

In [ ]:
#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [ ]:
products

In [ ]:
products['sentiment'].show()

# Train our sentiment classifier

In [ ]:
train_data,test_data = products.random_split(.8,seed=0)

In [ ]:
# Original Analysis Model 
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', 
                                                        features=['word_count'], 
                                                        validation_set=test_data)

In [ ]:
# Creating the model with selected words 
selected_words_model = turicreate.logistic_classifier.create(train_data,target='sentiment', 
                                                        features=features, 
                                                        validation_set=test_data)

## Analysing our model and weights 

In [ ]:
# Calling and descreibing our coefficients and weights allotted to each word
selected_words_model.coefficients.sort(key_column_names='value', ascending=True)

Out of the 11 words in selected_words, which one got the most positive weight? Which one got the most negative weight? 
Most Positive: love 
Most Negative: horrible

Makes total sense because love is a great word and horrible is a bad descriptor. 

## Evaluation of our models

In [ ]:
# Evaluate the orginal analysis model first 
sentiment_model.evaluate(test_data)

In [ ]:
# Evaluate the limited words model 
selected_words_model.evaluate(test_data)

Accuracy increases multifold! 

# Apply the sentiment classifier to better understand the Baby Trend Diaper Champ reviews

**Interpreting the difference in performance between the models:** To understand why the model with all word counts performs better than the one with only the selected_words, we will now examine the reviews for a particular product.

* We will investigate a product named ‘Baby Trend Diaper Champ’. (This is a trash can for soiled baby diapers, which keeps the smell contained.)

* Just like we did for the reviews for the giraffe toy in the Jupyter Notebook in the lecture video, before we start our analysis you should select all reviews where the product name is ‘Baby Trend Diaper Champ’. Let’s call this table diaper_champ_reviews.

* Again, just as in the video, use the sentiment_model to predict the sentiment of each review in diaper_champ_reviews and sort the results according to their ‘predicted_sentiment’.

In [ ]:
# Extract only the relevant data
diaper_champ_reviews = products[products['name']== 'Baby Trend Diaper Champ']
diaper_champ_reviews

In [ ]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

In [ ]:
diaper_champ_reviews['predicted_sentiment'] = selected_words_model.predict(diaper_champ_reviews, output_type = 'probability')

In [ ]:
products

# Sort the Diaper Champ reviews according to predicted sentiment

In [ ]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [ ]:
diaper_champ_reviews

In [ ]:
diaper_champ_reviews.tail()

## Show the most positive reviews

In [ ]:
diaper_champ_reviews[0]['review']

In [ ]:
diaper_champ_reviews[1]['review']

# Most negative reivews

In [ ]:
diaper_champ_reviews[-1]['review']

In [ ]:
diaper_champ_reviews[-2]['review']